<a href="https://colab.research.google.com/github/joyalkenus/Project_imageSearch/blob/main/run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This file is the major run file of this project
- Aims to give a user friendly interface to interact with the system
- aims to integrate the BERT LLM to the RAM model to achieve proper image search for the user description.

In [61]:
#@title Import dependencies
import ipywidgets as widgets
from IPython.display import clear_output, display, Image
import os


In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
!pip install transformers

In [64]:
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [65]:
#@title Install dependencies
!pip install timm transformers fairscale pycocoevalcap

clear_output()

In [66]:
# Download checkpoints
model_widget = widgets.Dropdown(
    options=["RAM", "Tag2Text"],
    value="RAM",
    description="Select mdoel:"
)
display(model_widget)

Dropdown(description='Select mdoel:', options=('RAM', 'Tag2Text'), value='RAM')

In [67]:
model = model_widget.value

In [68]:
def download_checkpoints(model):
    print('You selected', model)
    if not os.path.exists('pretrained'):
        os.makedirs('pretrained')

    if model == "RAM":
        ram_weights_path = 'pretrained/ram_swin_large_14m.pth'
        if not os.path.exists(ram_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/ram_swin_large_14m.pth -O pretrained/ram_swin_large_14m.pth
        else:
            print("RAM weights already downloaded!")
    else:
        tag2text_weights_path = 'pretrained/tag2text_swin_14m.pth'
        if not os.path.exists(tag2text_weights_path):
            !wget https://huggingface.co/spaces/xinyu1205/Recognize_Anything-Tag2Text/resolve/main/tag2text_swin_14m.pth -O pretrained/tag2text_swin_14m.pth
        else:
            print("Tag2Text weights already downloaded!")

download_checkpoints(model)
print(model, 'weights are downloaded!')

You selected RAM
RAM weights already downloaded!
RAM weights are downloaded!


In [69]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model1 = BertModel.from_pretrained('bert-base-uncased')


In [70]:
def encode_text(text):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        output = model1(**encoded_input)
    return output.last_hidden_state.mean(dim=1).numpy()




Use the Bert model to the capion data set



In [71]:
!python extract.py


/encoder/layer/0/crossattention/self/query is tied
/encoder/layer/0/crossattention/self/key is tied
/encoder/layer/0/crossattention/self/value is tied
/encoder/layer/0/crossattention/output/dense is tied
/encoder/layer/0/crossattention/output/LayerNorm is tied
/encoder/layer/0/intermediate/dense is tied
/encoder/layer/0/output/dense is tied
/encoder/layer/0/output/LayerNorm is tied
/encoder/layer/1/crossattention/self/query is tied
/encoder/layer/1/crossattention/self/key is tied
/encoder/layer/1/crossattention/self/value is tied
/encoder/layer/1/crossattention/output/dense is tied
/encoder/layer/1/crossattention/output/LayerNorm is tied
/encoder/layer/1/intermediate/dense is tied
/encoder/layer/1/output/dense is tied
/encoder/layer/1/output/LayerNorm is tied
--------------
pretrained/ram_swin_large_14m.pth
--------------
load checkpoint from pretrained/ram_swin_large_14m.pth
vit: swin_l
  Filename  ...                                 image_path
0    1.jpg  ...  /content/drive/My Drive

In [72]:
import pandas as pd
df = pd.read_csv('image_tags.csv')

In [73]:
df['Tags']

0    condiment | counter top | table | dinning tabl...
1    baseball hat | boy | child | floor | girl | ma...
Name: Tags, dtype: object

In [74]:
# Create text input widget
text_input = widgets.Text(
    description='Enter Text:',
    placeholder='Type something'
)
# Create a button widget
button = widgets.Button(description="Submit")

In [75]:
# Output widget to display results
output = widgets.Output()

In [76]:
def on_button_clicked(b):
    global user_input
    user_input = text_input.value  # Assign the input text to user_input variable
    with output:
        print(f"You entered: {user_input}")

# Bind the button click event to the function
button.on_click(on_button_clicked)
# Display the widgets
display(text_input, button, output)

Text(value='', description='Enter Text:', placeholder='Type something')

Button(description='Submit', style=ButtonStyle())

Output()

In [77]:
# Assume user_input is obtained from the widget
# user_input = "..."  # This is already set from the widget

# Encode the user input
user_embedding = encode_text(user_input)[0]

tags_string = df['Tags']

# Tokenize the string
encoded_input = tokenizer(tags_string, return_tensors='pt', padding=True, truncation=True, max_length=128)

# Assuming df['Tags'] contains the embeddings
embeddings = np.vstack(df['Tags'].values)

# Calculate cosine similarities
similarities = cosine_similarity([user_embedding], embeddings)

# Find the best match
best_match_index = similarities.argmax()
best_match_image_path = df.iloc[best_match_index]['image_path']

print(f"The most similar image is at: {best_match_image_path}")


ValueError: could not convert string to float: 'condiment | counter top | table | dinning table | plate | fill | food | fork | meal | noodle | peak | platter | salad | silverware | utensil | yellow'